In [15]:
from collections import deque
import numpy as np
from aocutils.grid import arr_to_dict

d = {'E': 1, 'W': -1, 'N': 1j, 'S': -1j}
turn = [complex(0.5,0.5), complex(-0.5,-0.5), complex(-0.5,0.5), complex(0.5,-0.5)]

def squeezein(node):
    global sq
    res = set()
    for wind in 'NS':
        if (dest := node + d[wind] + 0.5) in sq: res.add(dest)
        if (dest := node + d[wind] - 0.5) in sq: res.add(dest)
    for wind in 'EW':
        if (dest := node + d[wind] + 0.5j) in sq: res.add(dest)
        if (dest := node + d[wind] - 0.5j) in sq: res.add(dest)
    return list(res)

def squeezeout(node):
    global sq
    res = set()
    if sq[node] == '|':
        for i in [-0.5, 0.5]:
            if (dest := node + d['N'] + i) in grid : res.add(dest)
            if (dest := node + d['S'] + i) in grid : res.add(dest)
    if sq[node] == '-':
        for i in [-0.5j, 0.5j]:
            if (dest := node + d['W'] + i) in grid : res.add(dest)
            if (dest := node + d['E'] + i) in grid : res.add(dest)
    return list(res)

def neigh(node):
    global grid, distances
    if node not in sq:
        match grid[node]:
            case '|': return (node + d['N'], node + d['S'])
            case '-': return (node + d['W'], node + d['E'])
            case 'L': return (node + d['N'], node + d['E'])
            case 'J': return (node + d['N'], node + d['W'])
            case '7': return (node + d['S'], node + d['W'])
            case 'F': return (node + d['S'], node + d['E'])
            case '.': return tuple([node + direc for direc in d.values() if node + direc not in distances] + squeezein(node))
    else:
        match sq[node]:
            case '|': 
                res = [node + d['N'], node + d['S']] + [node + i for i in [0.5,-0.5] ] + [node + t for t in turn if node + t in sq]
            case '-': 
                res = [node + d['W'], node + d['E']] + [node + i for i in [0.5j,-0.5j]] + [node + t for t in turn if node+ t in sq]
        return tuple(res + squeezeout(node))

In [16]:
# parse grid with complex numbers
def solve(file, ch):
    global distances, sq, seen, grid
    f = open(file).read().split("\n")
    lines = [list(line) for line in f]
    lines = np.pad(lines,1)
    grid = arr_to_dict(lines, tocomplex=True)
    start = [k for k,v in grid.items() if v == 'S'].pop()
    grid[start] = ch
    def bfs(node):
        dis = 0
        tocheck = deque([node])
        while tocheck:
            for _ in range(len(tocheck)):
                cur = tocheck.popleft()
                for n in neigh(cur):
                    if n not in seen and (n in grid or n in sq):
                        seen.add(n)
                        distances[n] = dis + 1
                        tocheck.append(n)
            dis += 1
    bfs(start)
    print(max(distances.values()))
    border = {k for k,v in grid.items() if v == '0'}
    for k,v in grid.items():
        if grid != '0' and k not in distances:
            grid[k] = '.'

    for k,v in grid.items():
        if v not in '-FL' and grid.get(k + d['E'],'x') not in '7J-' :
            sq[k + complex(0.5, 0)] = '|'
        if v not in '7F|' and grid.get(k + d['S'],'x') not in 'LJ|' :
            sq[k + complex(0, -0.5)] = '-'
    for node in border:
        bfs(node)
    return len(set(grid) - set(distances) - seen)

# for file, ch in [('t0.txt', '7'), ('t1.txt', 'F'), ('in.txt', '|')]:
distances, seen , sq = {}, set(), {}
print(solve('in.txt', '|'))


6599
477


In [17]:
# original
def neigh(n):
    if grid[n] == '|':
        return [n+complex(0,1), n + complex(0,-1)]
    elif grid[n] == '-':
        return [n+complex(1,0), n + complex(-1,0)]
    elif grid[n] == 'L':
        return [n+complex(0,1), n + complex(1,0)]
    elif grid[n] == 'J':
        return [n+complex(0,1), n + complex(-1,0)]
    elif grid[n] == '7':
        return [n+complex(0,-1), n + complex(-1,0)]
    elif grid[n] == 'F':
        return [n+complex(0,-1), n + complex(1,0)]
    elif grid[n] == '.':
        print('error')
    elif grid[n] == 'S':
        return [n+complex(0,1), n+complex(0,-1)]
        # return [n+i for i in [complex(1,0), complex(-1,0), complex(0,1), complex(0,-1)] if grid.get(n+i,'.') != '.']
    else:
        print('error')

In [66]:
from aocutils.math import shoelace
shoelace(list(distances.keys()), include_border=False)

total for shoelace 7075.0


477.0

In [31]:
# original
def neigh(n):
    if grid[n] == '|':
        return [n+complex(0,1), n + complex(0,-1)]
    elif grid[n] == '-':
        return [n+complex(1,0), n + complex(-1,0)]
    elif grid[n] == 'L':
        return [n+complex(0,1), n + complex(1,0)]
    elif grid[n] == 'J':
        return [n+complex(0,1), n + complex(-1,0)]
    elif grid[n] == '7':
        return [n+complex(0,-1), n + complex(-1,0)]
    elif grid[n] == 'F':
        return [n+complex(0,-1), n + complex(1,0)]
    elif grid[n] == '.':
        print('error')
    elif grid[n] == 'S':
        return [n+complex(0,1), n+complex(0,-1)]
        # return [n+i for i in [complex(1,0), complex(-1,0), complex(0,1), complex(0,-1)] if grid.get(n+i,'.') != '.']
    else:
        print('error')
        
def neigh2(node):
    global safe, distances, sq
    res = set()
    n = node
    if node.real.is_integer() and node.imag.is_integer():
        # in grid
        for i in [complex(1,0), complex(-1,0), complex(0,1), complex(0,-1)]:
            if (node + i) in grid and (node+i) not in distances:
                res.add(node+i)
        for i in [complex(0.5,1), complex(-0.5,1), complex(0.5,-1), complex(-0.5,-1)]:
            # print(node+i,grid.get(node+i, 'x'))
            if grid.get(node+i, 'x') == '|':
                res.add(node+i)
        for i in [complex(1,0.5), complex(1,-0.5), complex(-1,0.5), complex(-1,-0.5)]:
            if grid.get(node+i, 'x') == '-':
                res.add(node+i)
    elif grid[node] == '|':
        # were in sq
        for i in [complex(0,1), complex(0,-1)]:
            if grid.get(node+i, 'x') == '|':
                res.add(node+i)
        for i in [complex(0.5,0.5), complex(-0.5,-0.5), complex(-0.5,0.5), complex(0.5,-0.5)]:
            if grid.get(node+i, 'x') == '-':
                res.add(node+i)
                
        for i in [complex(0.5,1), complex(-0.5,1), complex(0.5,-1), complex(-0.5,-1)]:
            if (node + i) in grid and (node+i) not in distances:
                res.add(node+i)
 
        
    elif grid[node] == '-':
        for i in [complex(1,0), complex(-1,0)]:
            if grid.get(node+i, 'x') == '-':
                res.add(node+i)
                
        for i in [complex(0.5,0.5), complex(-0.5,-0.5), complex(-0.5,0.5), complex(0.5,-0.5)]:
            if grid.get(node+i, 'x') == '|':
                res.add(node+i)
        for i in [complex(1,0.5), complex(1,-0.5), complex(-1,0.5), complex(-1,-0.5)]:
            if (node + i) in grid and (node+i) not in distances:
                res.add(node+i)
        
    return res

In [32]:
f = open('in.txt').read().split("\n")
lines = [list(line) for line in f]
lines = np.pad(lines,1)
grid = arr_to_dict(lines, tocomplex=True)
start = [k for k,v in grid.items() if v == 'S'].pop()

safe = set()
safeground = set()   
sq = {}
tiles = len(grid)

distances = {}
start = [k for k,v in grid.items() if v == 'S'].pop()
start
grid[start] = '|'
border = set()
seen = set()
def bfs(node):
    dis = 0
    tocheck = deque([node])
    while tocheck:
        for _ in range(len(tocheck)):
            cur = tocheck.popleft()
            # print(len(neigh2(cur)))
            for n in neigh(cur):
                if n not in seen:
                    seen.add(n)
                    distances[n] = dis + 1
                    tocheck.append(n)
        dis += 1
bfs(start)


loop = len(distances)        
def bfs(node):
    dis = 0
    tocheck = deque([node])
    while tocheck:
        for _ in range(len(tocheck)):
            cur = tocheck.popleft()
            # print(len(neigh2(cur)))
            for n in neigh2(cur):
                if n not in seen:
                    safe.add(n)
                    seen.add(n)
                    tocheck.append(n)
        dis += 1
[(k,v) for k,v in distances.items() if v == max(distances.values())]
print(len(grid), len(distances), max(distances.values()))

for k,v in grid.items():
    if grid[k] != '0' and k not in distances:
        grid[k] = '.'
        
real = len(lines[0])
imag = len(lines)
for r in range(real):
    for i in range(0,-imag,-1):
        # print(r,c)
        if (grid.get(complex(r,i), 'x')) == 'x':
            print('error cannot be found')
            sys.exit()
            

        if grid.get(complex(r,i), 'x') not in list('-FL') and grid.get(complex(r+1,i),'x') not in list('7J-') and (complex(r,i) in distances or complex(r+1,i) in distances):
            # pipe N S
            sq[complex(r+0.5, i)] = '|'
        if grid.get(complex(r,i), 'x') not in list('7F|') and grid.get(complex(r,i-1),'x') not in list('LJ|')  and (complex(r,i) in distances or complex(r,i-1) in distances):
            # pipe WE
            sq[complex(r,i-0.5)] = '-'
grid |= sq

for node in grid:
    if grid[node] == '0':
        border.add(node)
for node in border:
    bfs(node)
left = (set(grid) - set(sq) - set(distances) - safe)
len(left)

20164 13198 6599
